In [1]:
%%bash
echo "deb [ arch=amd64 ] https://downloads.skewed.de/apt bionic main" >> /etc/apt/sources.list
apt-key adv --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
apt-get update
apt-get install python3-graph-tool

Executing: /tmp/apt-key-gpghome.OtSv5box46/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,568 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [1,082 kB]
Get:11 

gpg: key 612DEFB798507F25: public key "Tiago de Paula Peixoto <tiago@skewed.de>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [ ]:
#splitted
import pandas as pd
edges = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/splitted_unidata_sub/edges.parquet')
edges_test = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/splitted_unidata_sub/edges_test.parquet')

In [2]:

import pandas as pd
edges = pd.concat([
    pd.read_parquet(f'/content/drive/MyDrive/ColabNotebooks/unidata_sub/edges_sub_frag_{i}.parquet') for i in range (5) 
    ])
edges_test = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/unidata_sub/edges_test_sub.parquet')

vertices = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/unidata_sub/train_vprops')
vertices_test = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/unidata_sub/test_vprops')

In [3]:
JOIN = True

In [ ]:
JOIN = False

In [4]:
#Params
TEST_CLICK_WEIGHT = 3
TEST_CART_WEIGHT = 6
TEST_ORDER_WEIGHT_L= 6
TEST_ORDER_WEIGHT_S = 2


MIN_TOPK_COUSER = 500
COFITNESS_THRES = 0


ALPHA = 0.5
BETA = 0.5


# Create edges and vertices

In [5]:
all_edges = pd.concat((edges, edges_test))

In [6]:
del edges, edges_test

In [7]:
all_vertices = pd.concat((vertices, vertices_test))

In [8]:
del vertices, vertices_test

In [9]:
all_edges

,"('session', '')","('aid', '')","('ts', 'last')","('ts', 'first')","('clicks', 'sum')","('carts', 'sum')","('orders', 'sum')","('subsession', 'encode')"
0,0,6851,2022018,2022018,1,0,0,35184372088832
1,0,7563,2031353,2031315,2,0,0,70368744177664
2,0,10268,1853151,1853151,1,0,0,2199023255552
3,0,16246,62919,62919,1,0,0,2
4,0,30373,1798887,1798887,1,0,0,1099511627776
...,...,...,...,...,...,...,...,...
5006552,14571577,1141710,3023974,3023974,1,0,0,1
5006553,14571578,519105,3023975,3023975,1,0,0,1
5006554,14571579,739876,3023975,3023975,1,0,0,1
5006555,14571580,202353,3023981,3023981,1,0,0,1


In [10]:
all_edges.iloc[:,1] = all_edges.iloc[:,1] + 100000000


In [11]:
all_vertices = all_vertices.sort_values('session')

In [12]:
all_vertices = all_vertices.values

In [13]:
import numpy as np
all_edges = all_edges.values

# Load to graph

In [14]:
import graph_tool

In [15]:
from graph_tool import Graph
ug = Graph(directed=False)

In [16]:
vmap = ug.add_edge_list(all_edges[:, : 2].astype(np.uint32))

In [17]:
data_type = {
    'elts': 'unsigned int',
    'efts': 'unsigned int',
    'eclick': 'short',
    'ecart': 'short',
    'eorder': 'short',
    'esub': 'unsigned long'
}

In [18]:
for i, prop in enumerate(['elts', 'efts', 'eclick', 'ecart', 'eorder', 'esub']):
  setattr(ug.ep, prop, ug.new_ep(data_type[prop], vals = all_edges[:, i + 2]))  


In [19]:
del all_edges

In [20]:
ug.num_edges()

138610598

In [21]:
elts = ug.ep['elts']
efts = ug.ep['efts']
eclick = ug.ep['eclick']
ecart = ug.ep['ecart']
eorder = ug.ep['eorder']
esub = ug.ep['esub']


In [22]:
reverse_variety = ug.new_vp('double',  1 / (1 + np.log(all_vertices[:, 1] + 1)) )
consistency = ug.new_vp('double',  1 / (1 + np.log(all_vertices[:, 2] + 1)) )

In [23]:
del all_vertices

# Load test

In [24]:
ROOT = '/content/drive/MyDrive/ColabNotebooks/điệp viên'
DATA_FOLDER = ROOT + '/data'



In [25]:
# ROOT = '/content/drive/MyDrive/ColabNotebooks/điệp viên'
# DATA_FOLDER = ROOT + '/splitted_data'



In [26]:
import pandas as pd
test_df = pd.read_parquet(DATA_FOLDER + '/test.parquet')

In [27]:
session_list = np.sort(test_df['session'].unique()).tolist()


In [28]:
# session_list = session_list[:len(session_list) // 100]

In [29]:
del test_df

# Find neighbour strategy

In [30]:
#PARAMS

get_items_params = {
      'test_click_weight': TEST_CLICK_WEIGHT,
      'test_cart_weight': TEST_CART_WEIGHT,
      'test_order_weight_l': TEST_ORDER_WEIGHT_L,
      'test_order_weight_s': TEST_ORDER_WEIGHT_S,
  }

get_items_click_params = {
      'click_weight': 6,
      'cart_weight' : 3, 
      'order_weight': 1, 
      'day' : 8 * 3600,  
    }

get_items_cart_params = {
      'click_weight': 3, 
      'cart_weight' : 6, 
      'order_weight': 1, 
      'day' : 8 * 3600,   
    }
get_items_order_params = {
      'click_weight': 3, 
      'cart_weight' : 10, 
      'order_weight': 1,
      'day' : 24 * 7 * 3600,   
    }

In [31]:


def get_test_items(user, beta, action):
  global ug
  #0: src, 1: target, 2: lts, 3: eclick, 4: ecart, 5:eorder
  edges = ug.get_all_edges(user, eprops = [elts, eclick, ecart, eorder])
        
  return _get_test_items(edges, beta, **get_items_params)


def get_user_neighbours(item):
  global ug
  #1 target 2 ltss 3 ftss 4 click 5 cart 6 order 7 subsession
  edges = ug.get_all_edges(item, eprops = [elts, efts, eclick, ecart, eorder, esub])
  user_props = ug.get_all_neighbours(item, vprops = [reverse_variety, consistency]) 
  return edges[:,1], edges[:,2], edges[:,3], edges[:, 4], edges[:, 5], edges[:, 6], edges[:, 7], user_props[:, 1] * user_props[:, 2] 

def get_item_neighbours(user, lts, fts, prevsub, prev_click, prev_cart, prev_order, alpha, action):
  global ug
  #edge idx: 0 is src, 1 is target, 2 is last time stamp, 3 is first time stamp, 4 click, 5 cart, 6 order, 7 subsession

  edges = ug.get_all_edges(user, eprops = [elts, efts, eclick, ecart, eorder, esub])

  if action == 0:
    args = get_items_click_params
  
  if action == 1:
    args = get_items_cart_params
  
  if action == 2:
    args = get_items_order_params
  
  return _get_item_neighbours(edges, lts, fts, prev_click = prev_click, prev_cart = prev_cart, prev_order = prev_order
                              , alpha = alpha, prevsub = prevsub, **args)
  


# Numba funcs

In [32]:
import numba as nb
import numpy as np

@nb.njit(nb.int64[:](nb.float64[:]), cache = True)
def get_rank(x):
  return np.argsort(np.argsort(x))

@nb.njit(nb.int64(nb.int64), cache = True)
def decode_1d(x):
  rs = 0
  while x > 0:
    if x % 2 == 1:
      rs += 1
    x = x // 2
  return rs

@nb.njit(nb.int64[:](nb.int64[:]), cache = True)
def decode_2d(x):
  return np.array([decode_1d(i) for i in x])
    
@nb.njit(nb.float64[:](nb.int64[:]), cache = True)
def normalize(X):
  m = np.max(X)
  return (m - X) / (m - np.min(X) + 1e-12) 

@nb.njit([nb.float64[:](nb.float64[:], nb.float64, nb.float64),
          nb.float64[:](nb.float64[:], nb.int64, nb.float64),
          nb.float64[:](nb.float64[:], nb.float64, nb.int64),
         nb.float64[:](nb.float64[:], nb.int64, nb.int64)], cache = True)
def normalize_to_range(X, l, r):
  m = np.min(X)
  return (X - m) / (np.max(X) - m + 1e-12) * (r -l) + l


@nb.njit([nb.types.UniTuple(nb.float64[:],2) (nb.float64[:], nb.float64[:]),
          nb.types.Tuple((nb.int64[:], nb.float64[:])) (nb.int64[:], nb.float64[:]),
          nb.types.Tuple((nb.int64[:], nb.int64[:])) (nb.int64[:], nb.int64[:]),
          ], cache = True)
def sum_by_group(groups, values):
    order = np.argsort(groups)
    groups = groups[order]
    values = values[order]
    values = np.cumsum(values)
    index = np.ones(len(groups), 'bool')
    index[:-1] = groups[1:] != groups[:-1]
    values = values[index]
    groups = groups[index]
    values[1:] = values[1:] - values[:-1]
    return groups, values



@nb.njit([
          nb.types.Tuple((nb.int64[:], nb.float64[:], nb.float64[:])) (nb.int64[:], nb.float64[:], nb.float64[:]),
          ], cache = True)
def sum_by_group_2_values(groups, values, values1):
    order = np.argsort(groups)

    groups = groups[order]
    values = values[order]
    values = np.cumsum(values)
    values1 = values1[order]
    values1 = np.cumsum(values1)

    index = np.ones(len(groups), 'bool')
    index[:-1] = groups[1:] != groups[:-1]

    values = values[index]
    values1 = values1[index]

    groups = groups[index]
    values[1:] = values[1:] - values[:-1]
    values1[1:] = values1[1:] - values1[:-1]
    return groups, values, values1


@nb.njit([nb.types.UniTuple(nb.float64[:], 2) (nb.float64[:], nb.float64[:], nb.float64[:]),
          nb.types.Tuple((nb.int64[:], nb.float64[:])) (nb.int64[:], nb.float64[:], nb.int64[:]),
          nb.types.Tuple((nb.int64[:], nb.int64[:])) (nb.int64[:], nb.int64[:], nb.int64[:]),
          ], cache = True)
def sum_by_group_exclude(groups, values, exclude):
    inclusive = np.ones(len(groups), 'bool')
    for e in exclude:
      inclusive = inclusive & (groups != e)
    
    return  sum_by_group(groups[inclusive], values[inclusive])
    
@nb.njit([
          nb.types.Tuple((nb.int64[:], nb.float64[:], nb.float64[:])) (nb.int64[:], nb.float64[:], nb.float64[:], nb.int64[:]),
          ], cache = True)
def sum_by_group_exclude_2_values(groups, values, values1, exclude):
    inclusive = np.ones(len(groups), 'bool')
    for e in exclude:
      inclusive = inclusive & (groups != e)
    
    return  sum_by_group_2_values(groups[inclusive], values[inclusive], values1[inclusive])
    
@nb.njit([
          nb.types.Tuple((nb.int64[:], nb.float64[:], nb.float64[:]))(nb.int64[:,:], nb.int64, nb.int64, nb.int64, nb.int64, nb.float64, nb.int64, nb.float64, nb.float64, nb.float64 , nb.float64, nb.int64), 
          nb.types.Tuple((nb.int64[:], nb.float64[:], nb.float64[:]))(nb.int64[:,:], nb.int64, nb.int64, nb.int64, nb.int64, nb.int64, nb.int64, nb.float64, nb.int64, nb.int64 , nb.int64, nb.int64), 
          ], cache = True)
def _get_item_neighbours(edges, lts, fts, prevsub, prev_click, prev_cart, prev_order, alpha, click_weight, cart_weight, order_weight, day):
  #edge idx: 0 is src, 1 is target, 2 is last time stamp, 3 is first time stamp, 4 click, 5 cart, 6 order, 7 -> subsession

  cosubsession = prevsub & edges[:, 7]
  cosubsession = decode_2d(cosubsession)
  
  cosubsession = np.clip(cosubsession, 0, 1)

  # coclick = np.where(edges[:,4] < prev_click, edges[:,4], prev_click)
  # cocart = np.where(edges[:,5] < prev_cart, edges[:,5], prev_cart)
  # coorder = np.where(edges[:,6] < prev_order, edges[:,6], prev_order)
  coclick = np.minimum(edges[:, 4], prev_click)
  cocart = np.minimum(edges[:, 5], prev_cart)
  coorder = np.minimum(edges[:, 6], prev_order)


  weight = coclick * click_weight + cocart * cart_weight  + coorder * order_weight 


  #check if union, if yes time diff = 0, else = min(fts1 - lts2, fts2 - lts1)
  is_end_later = edges[:, 2] > lts

  is_union = np.empty(edges.shape[0], dtype = np.bool_)
  is_union[is_end_later] =  edges[is_end_later, 3] < lts
  is_end_sooner = ~is_end_later
  is_union[is_end_sooner] = edges[is_end_sooner, 2] > fts

  subsession_coef = 1.5 ** cosubsession

  time_diff = np.empty(edges.shape[0],  dtype = np.float32) 
  
  time_diff[is_union] = 0

  not_union_and_later = ~is_union & is_end_later
  not_union_and_sooner = ~is_union & is_end_sooner

  time_diff[not_union_and_later] = np.abs(lts - edges[not_union_and_later, 3]) / day
  time_diff[not_union_and_sooner] = np.abs(fts - edges[not_union_and_sooner, 2]) / day
  time_decay = alpha ** time_diff


  fitness_subco = weight  * subsession_coef 

  fitness_time_decay = weight * time_decay 
  return sum_by_group_2_values(edges[:, 1], fitness_subco, fitness_time_decay)
  


@nb.njit(nb.types.Tuple((nb.int64[:], nb.float64[:]))(nb.int64[:, :], nb.float64, nb.float64, nb.float64, nb.float64, nb.float64) , cache = True)
def _get_test_items(edges, beta, test_click_weight, test_cart_weight, test_order_weight_l, test_order_weight_s):
  #0: src, 1: target, 2: lts, 3: eclick, 4: ecart, 5:eorder
  
  time_decay = beta ** normalize(edges[:, 2])

  order_weight = np.where(edges[:, 5] > 1, test_order_weight_l, test_order_weight_s)

  weight = edges[:, 3] * test_click_weight + edges[:, 4] * test_cart_weight + order_weight 

  fitness =  time_decay * weight

  order = np.argsort(- fitness )  

  return edges[order, 1], fitness + order_weight / 2
  


@nb.njit([
          nb.types.Tuple((nb.int64[:], nb.float64[:], nb.int64[:], nb.boolean[:])) (nb.int64[:], nb.float64[:], nb.int64, nb.float64),
          nb.types.Tuple((nb.int64[:], nb.int64[:], nb.int64[:], nb.boolean[:])) (nb.int64[:], nb.int64[:], nb.int64, nb.float64),
          # nb.types.Tuple((nb.int64[:], nb.int64[:], nb.int64[:], nb.boolean[:])) (nb.float64[:], nb.float64[:], nb.int64, nb.float64),
          ], cache = True)
def find_topk_user(cousers, cofitness, min_topk_user, cofitness_thres):
  #sort cousers ascending by id
  order = np.argsort(cousers)
  cousers = cousers[order]
  cofitness = cofitness[order]

  #calculate groupby sum of cousers
  cumcofitness = np.cumsum(cofitness)

  #find unique
  index = np.ones(len(cousers), 'bool')

  #this is the last index of each group
  index[:-1] = cousers[1:] != cousers[:-1]
  cumcofitness = cumcofitness[index]
  

  #find start and end of groups
  nodes = np.hstack((np.array([0]), np.where(index)[0] + 1))
    
  #caluculate groupby sum of cousers
  cumcofitness[1:] = cumcofitness[1:] - cumcofitness[:-1]
  order_fit = np.argsort(-cumcofitness)

  k = min_topk_user
  if cofitness_thres > 0:
    #get topk user
    #dynamic top user
    total_fitness = np.sum(cumcofitness)
    
    k = 0
    remain = 0
    thresh_hold = cofitness_thres * total_fitness
    while remain < thresh_hold:
      remain += cumcofitness[order_fit][k]
      k+=1 
    
    if k < min_topk_user:
      k = min_topk_user

  
  order_fit = order_fit[:k]

  starts = nodes[order_fit]
  ends = nodes[order_fit + 1]
  
  filter = np.zeros(len(cousers), 'bool')
  for i in range(len(starts)):
    filter[starts[i] : ends[i]] = 1
  
  return cousers[filter], cofitness[filter], order, filter




# Step

In [33]:
def get_type_items(user, clicks, carts, orders, len_clicks, len_carts, len_orders, action):

  #get item in tests

  items, fitness = get_test_items(user, BETA, action = action)
  
  if action == 0:
    lens = len_clicks
    preds = clicks
  if action == 1:
    lens = len_carts
    preds = carts
  if action == 2:
    lens = len_orders
    preds = orders



  # lens.append(len(items))

  if len(items)  > 19:
    preds.append(items.tolist()[:20])
  
  else:
    
    num_more = 20 - len(items)
    
    
    
    #get couser
    coalls = [get_user_neighbours(items[i]) for i in range(len(items))]

    cofitness = np.concatenate([all[6] * fitness[i] for i, all in enumerate(coalls)] )

    #find topk cousers
    cousers, cofitness, order, filter  = find_topk_user(np.concatenate([all[0] for all in coalls]), cofitness, MIN_TOPK_COUSER, COFITNESS_THRES)
    coltss = np.concatenate([all[1] for all in coalls])[order][filter]
    coftss = np.concatenate([all[2] for all in coalls])[order][filter]
    coclick = np.concatenate([all[3] for all in coalls])[order][filter]
    cocart = np.concatenate([all[4] for all in coalls])[order][filter]
    coorder = np.concatenate([all[5] for all in coalls])[order][filter]
    cosubsession = np.concatenate([all[6] for all in coalls])[order][filter]
    

    #get coitems
    coalls = [get_item_neighbours(cousers[i], coltss[i], coftss[i], cosubsession[i], coclick[i], cocart[i], coorder[i], ALPHA, action = action) for i in range(len(cousers))]

    #multiply item by fitness of user which interacted with it
    cofitness_cosub = np.concatenate([all[1] * cofitness[i] for i, all in enumerate(coalls)]) 
    cofitness_time_decay = np.concatenate([all[2] * cofitness[i] for i, all in enumerate(coalls)]) 
    coitems = np.concatenate([all[0] for all in coalls])
    
    #count covisit
    coitems, cofitness_cosub, cofitness_time_decay= sum_by_group_exclude_2_values(coitems, cofitness_cosub, cofitness_time_decay, exclude =  items)
    
    #sort by count then to fitness

    rank_cosub = get_rank(cofitness_cosub)
    rank_time_decay = get_rank(cofitness_time_decay)


    order = np.argsort( -(rank_cosub  + rank_time_decay) )
    
    news = items.tolist() +  coitems[order[:num_more]].tolist()

    del cousers, coitems, coalls, cofitness_cosub, cofitness_time_decay, coltss, coftss, coclick, cocart
    
    preds.append(news)

    


# Test pipeline

In [ ]:
from tqdm.notebook import tqdm
clicks = []
carts = []
orders = []
len_clicks = []
len_carts = []
len_orders = []
for user in tqdm(session_list[:100]):
# for user in tqdm([0]):
    for i in range(3):
      get_type_items(user, clicks, carts, orders, len_clicks, len_carts, len_orders, i)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
clicks

[[100059625,
  100941596,
  101340695,
  100601769,
  101130911,
  101657590,
  100110837,
  100426813,
  100251302,
  100171014,
  100025820,
  100695253,
  100631502,
  100662942,
  100513418,
  100390074,
  101574761,
  100850964,
  101430859,
  100479009],
 [101142000,
  100736515,
  100973453,
  100582732,
  100322370,
  100481350,
  101758603,
  100300793,
  100231487,
  100866199,
  101776422,
  100913104,
  100321547,
  101109973,
  101552037,
  100064218,
  101224420,
  100284207,
  101497474,
  100345087],
 [100199008,
  100918667,
  100194067,
  100057315,
  100141736,
  101538134,
  100662349,
  101460571,
  100620545,
  100624327,
  100039255,
  100617566,
  101289390,
  101426624,
  101193648,
  101604481,
  101448389,
  100424487,
  100134740,
  100123224],
 [100779477,
  100834354,
  100476063,
  100595994,
  100889671,
  100975116,
  100562753,
  100740494,
  100987399,
  101344773,
  100413962,
  101494780,
  100229748,
  100127404,
  101711180,
  100829180,
  1016694

# Multiprocessing

In [34]:
def shuffle(l):  
    perm = np.random.permutation(len(l))  
    l[:] = [l[j] for j in perm]  
    return perm


def unshuffle(L, perm):  
    
    res = [[None] * len(l) for l in L]
    for i, j in enumerate(perm):
        for k, l in enumerate(L):
          res[k][j] = l[i]
          
    for i, l in enumerate(L):
      l[:] = res[i]

In [35]:
from tqdm import tqdm
import multiprocessing

def batch_proc(proc_num, batch, return_dict):
  clicks = []
  carts = []
  orders = []
  len_clicks = []
  len_carts = []
  len_orders = []
  for user in tqdm(batch):
    for i in range(3):
      get_type_items(user, clicks, carts, orders, len_clicks, len_carts, len_orders, i)

  return_dict[proc_num] =  clicks, carts, orders, len_clicks, len_carts, len_orders
    

In [36]:
perm = shuffle(session_list)

In [37]:
#divide into batches
# NUM_CPU = 30
NUM_CPU = multiprocessing.cpu_count()
batch_size = len(session_list) // NUM_CPU
start = np.arange(NUM_CPU) * batch_size
end = start + batch_size
end[-1] = len(session_list)
batches = [session_list[start[i] : end[i]] for i in range(len(start))]

In [38]:
manager = multiprocessing.Manager()
return_dict = manager.dict()
jobs = [multiprocessing.Process(target=batch_proc, args=(i, batch, return_dict)) for i, batch in enumerate(batches)]
len(jobs)

40

In [ ]:
%%time
for job in jobs:
  job.start()
for job in jobs:
  job.join()


 46%|████▌     | 19093/41795 [1:19:35<1:19:20,  4.77it/s]

In [ ]:
clicks = []
carts = []
orders = []
len_clicks = []
len_carts = []
len_orders = []
from tqdm.notebook import tqdm
for i in tqdm(range(len(return_dict))):
    clicks.extend(return_dict[i][0])
    carts.extend(return_dict[i][1])
    orders.extend(return_dict[i][2])
    len_clicks.extend(return_dict[i][3])
    len_carts.extend(return_dict[i][4])
    len_orders.extend(return_dict[i][5])


In [ ]:
del return_dict

In [ ]:
unshuffle([clicks,carts,orders,len_clicks,len_carts,len_orders, session_list], perm)


# Create submission

In [ ]:
count = 0
num_all = 0
missing = []
for preds in [clicks, carts, orders]:
  for i, p in enumerate(preds):
    if len(p) != 20:
      missing.append(i)
      count+= 20 - len(p)
    num_all +=1

len(missing), count, num_all

In [ ]:
click_new_append = [' '.join([str(int(i) - 100000000) for i in a]) for a in clicks] if JOIN else [[int(j) - 100000000 for j in i] for i in clicks]
cart_new_append = [' '.join([str(int(i) - 100000000) for i in a]) for a in carts] if JOIN else [[int(j) - 100000000 for j in i] for i in carts]
order_new_append = [' '.join([str(int(i) - 100000000) for i in a]) for a in orders] if JOIN else [[int(j) - 100000000 for j in i] for i in orders]

In [ ]:
submission1 = pd.DataFrame({'session_type': session_list, 'labels': click_new_append})
submission2 = pd.DataFrame({'session_type': session_list, 'labels': cart_new_append})
submission3 = pd.DataFrame({'session_type': session_list, 'labels': order_new_append})

In [ ]:
submission1['session_type'] = submission1['session_type'].apply(lambda x: str(x) + '_clicks')
submission2['session_type'] = submission2['session_type'].apply(lambda x: str(x) + '_carts')
submission3['session_type'] = submission3['session_type'].apply(lambda x: str(x) + '_orders')

In [ ]:
submission1

In [ ]:
final = pd.concat([submission1, submission2, submission3])

In [ ]:
del submission1, submission2, submission3

In [ ]:
final

# Eval valid

In [ ]:
import pandas as pd

def get_score(df, split = False):
    submission = df.copy()
    test_labels = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/điệp viên/splitted_data/test_labels.parquet')
    test_labels = test_labels[test_labels['session'] <= np.amax(session_list)]
    
    submission['session'] = submission.session_type.apply(lambda x: int(x.split('_')[0]))
    
    session = set(test_labels['session'].unique().tolist())
    for j in session_list:
      assert j in session


    submission['type'] = submission.session_type.apply(lambda x: x.split('_')[1])
    
    if split:
      submission.labels = submission.labels.apply(lambda x: [int(i) for i in x.split(' ')[:20]])

    test_labels = test_labels.merge(submission, how='left', on=['session', 'type'])
    test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
    test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)

    del submission

    recall_per_type = test_labels.groupby(['type'])['hits'].sum() / test_labels.groupby(['type'])['gt_count'].sum() 

    print (f"Score : {(recall_per_type * pd.Series({'clicks': 0.10, 'carts': 0.30, 'orders': 0.60})).sum()}")
    return recall_per_type

In [ ]:
get_score(final, JOIN)

Score : 0.5068532886454155


type
carts     0.351505
clicks    0.480885
orders    0.588855
dtype: float64

# Save sumission

In [ ]:
final.to_csv('/content/drive/MyDrive/ColabNotebooks/12_9_4_submission.csv', index = False)

#Submit


In [ ]:
!pip install kaggle

In [ ]:
%%bash
rm ~/.kaggle/kaggle.json
mkdir ~/.kaggle
cp /content/drive/MyDrive/ColabNotebooks/kaggle.json ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions submit -c otto-recommender-system -f /content/drive/MyDrive/ColabNotebooks/12_9_4_submission.csv -m "cosub"

#Submit 1

In [ ]:
!pip install kaggle

In [ ]:
%%bash
rm ~/.kaggle/kaggle.json
mkdir ~/.kaggle
cp /content/drive/MyDrive/ColabNotebooks/kaggle1.json ~/.kaggle
mv ~/.kaggle/kaggle1.json ~/.kaggle/kaggle.json
chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions submit -c otto-recommender-system -f /content/drive/MyDrive/ColabNotebooks/12_9_2_submission.csv -m "validate mega graph tool test sampling"

In [ ]:
preds[0]

In [ ]:
count = 0
missing = []
for i, p in enumerate(preds):
  if len(p) != 20:
    missing.append(i)

count

In [ ]:
new_append = [' '.join([str(int(j) - 100000000) for j in i) for m, i in enumerate(preds)] if JOIN else [[int(j) - 100000000 for j in i[ : lens[m]]] for m, i in enumerate(preds)]

In [ ]:
submission1 = test_df.copy()
submission1['labels'] = new_append
submission2 = submission1.copy()
submission3 = submission1.copy()

In [ ]:
submission1

In [ ]:
submission1['session_type'] = submission1['session'].apply(lambda x: str(x) + '_clicks')
submission2['session_type'] = submission1['session'].apply(lambda x: str(x) + '_carts')
submission3['session_type'] = submission1['session'].apply(lambda x: str(x) + '_orders')

In [ ]:
final = pd.concat([submission1, submission2, submission3])

In [ ]:
del submission1, submission2, submission3

In [ ]:
del final['session'], final['ts']


In [ ]:
final = final.droplevel(1, axis = 1)

In [ ]:
final

In [ ]:
final.to_csv('/content/drive/MyDrive/ColabNotebooks/12_9_3_submission.csv', index = False)

# Create submission test only 

In [ ]:
count = 0
num_all = 0
missing = []
for preds in [clicks, carts, orders]:
  for i, p in enumerate(preds):
    if len(p) != 20:
      missing.append(i)
      count+= 20 - len(p)
    num_all +=1

len(missing), count, num_all

(90, 1689, 5015409)

In [ ]:
click_new_append = [' '.join([str(int(i) - 100000000) for i in a[ : len_clicks[m]]]) for m, a in enumerate(clicks)] if JOIN else [[int(j) - 100000000 for j in i[ : len_clicks[m]]] for m, i in enumerate(clicks)]
cart_new_append = [' '.join([str(int(i) - 100000000) for i in a[ : len_carts[m]]]) for m, a in enumerate(carts)] if JOIN else [[int(j) - 100000000 for j in i[ : len_carts[m]]] for m, i in enumerate(carts)]
order_new_append = [' '.join([str(int(i) - 100000000) for i in a[ : len_orders[m]]]) for m, a in enumreate (orders)] if JOIN else [[int(j) - 100000000 for j in i[ : len_orders[m]]] for m, i in enumerate(orders)]

NameError: ignored

In [ ]:
submission1 = pd.DataFrame({'session_type': session_list, 'labels': click_new_append})
submission2 = pd.DataFrame({'session_type': session_list, 'labels': cart_new_append})
submission3 = pd.DataFrame({'session_type': session_list, 'labels': order_new_append})

In [ ]:
submission1['session_type'] = submission1['session_type'].apply(lambda x: str(x) + '_clicks')
submission2['session_type'] = submission2['session_type'].apply(lambda x: str(x) + '_carts')
submission3['session_type'] = submission3['session_type'].apply(lambda x: str(x) + '_orders')

In [ ]:
submission1

In [ ]:
final = pd.concat([submission1, submission2, submission3])

In [ ]:
del submission1, submission2, submission3

In [ ]:
final

# Save sumission

In [ ]:
final.to_csv('/content/drive/MyDrive/ColabNotebooks/12_9_5_submission.csv', index = False)